In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('data.csv')

In [ ]:
# replace missing datda
data.replace('?', np.nan, inplace=True)
data=data.astype({"age": np.float64, "fare": np.float64})

In [ ]:
# Visuzalizing variabe relationships
import seaborn as sns
import matplotlib.pyplot as plt

fig, axs = plt.subplots(ncols=5, figsize=(30,5))
sns.violinplot(x="survived", y="age", hue="sex", data=data, ax=axs[0])
sns.pointplot(x="sibsp", y="survived", hue="sex", data=data, ax=axs[1])
sns.pointplot(x="parch", y="survived", hue="sex", data=data, ax=axs[2])
sns.pointplot(x="pclass", y="survived", hue="sex", data=data, ax=axs[3])
sns.violinplot(x="survived", y="fare", hue="sex", data=data, ax=axs[4])

In [ ]:
# check for correlations in the data
# all var's need to be converted to numeric
data.replace({'male': 1, 'female':0}, inplace=True)
# correlate the relationship between all variables and survival.
data.corr().abs()[['survived']]

In [ ]:
# Hypothesize that sibsp and parch are related and affect survivability, and group them into a new column called "relatives" to see whether the combination of them has a higher correlation to survivability. 
data['relatives'] = data.apply(lambda row: int((row['sibsp'] + row['parch']) >0), axis=1)
data.corr().abs()['survived']

In [ ]:
# Given that a person with relatives have a higher correlation of surviving then those w/o, we can drop the low corr calues
# of sibsp and parch. 
# We will also be dropping all NaN values
data = data[['sex','pclass','age','relatives','fare','survived']].dropna()

In [ ]:
# Train and evaluate the model

#Split the data into training and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data[['sex','pclass','age','relatives','fare']], data.survived,
test_size=0.2, random_state=0)

In [ ]:
# Now we need to normalize the features so they are all treated equally. For example, within the dataset the values for age range from ~0-100, while gender is only a 1 or 0.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.fit_transform(x_test)

In [ ]:
# Model selection
# We wil use Naives Bayes, a common algo for classification problems
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)
print(model.fit(X_train, y_train))

In [ ]:
# Test the model against the test set
from sklearn import metrics
predict_test = model.predict(X_test)
print(metrics.accuracy_score(y_test, predict_test))

In [39]:
# Run the same procedure using Neural Networks instead. Unlike the machine learning algorithm you looked at previously, neural networks are a form of deep learning wherein you don't need to know an ideal algorithm for your problem set ahead of time. 

from keras.models import Sequential
from keras.layers import Dense

model = Sequential()



In [40]:
# add the layers of the neural network. For now, let's keep things simple and just use three layers. Add the following code to create the layers of the neural network.
model.add(Dense(5, kernel_initializer= 'uniform', activation='relu', input_dim = 5)) # Layer 1
model.add(Dense(5, kernel_initializer= 'uniform', activation='relu')) # Layer 2
model.add(Dense(1, kernel_initializer= 'uniform', activation='sigmoid')) # Layer 3

# the sigmoid activation function is required for the final layer as the output you want (of whether a passenger survives or not) needs to be scaled in the range of 0-1 (the probability of a passenger surviving).

model.summary()


Model: &quot;sequential_1&quot;
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 5)                 30        
_________________________________________________________________
dense_16 (Dense)             (None, 5)                 30        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 6         
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Next we need to define the type of optimizer to use, how to calculate loss and what metric to optimize

model.compile(optimizer="adam", loss = "binary_crossentropy", metrics=['accuracy'])
model.fit(X_train,y_train, batch_size=32, epochs=50)


Epoch 1/50
27/27 [==============================] - 0s 2ms/step - loss: 0.6921 - accuracy: 0.5885
Epoch 2/50
27/27 [==============================] - 0s 2ms/step - loss: 0.6898 - accuracy: 0.5873
Epoch 3/50
27/27 [==============================] - 0s 1ms/step - loss: 0.6847 - accuracy: 0.6914
Epoch 4/50
27/27 [==============================] - 0s 1ms/step - loss: 0.6736 - accuracy: 0.7715
Epoch 5/50
27/27 [==============================] - 0s 1ms/step - loss: 0.6553 - accuracy: 0.7727
Epoch 6/50
27/27 [==============================] - 0s 1ms/step - loss: 0.6351 - accuracy: 0.7620
Epoch 7/50
27/27 [==============================] - 0s 1ms/step - loss: 0.6157 - accuracy: 0.7644
Epoch 8/50
27/27 [==============================] - 0s 2ms/step - loss: 0.6016 - accuracy: 0.7667
Epoch 9/50
27/27 [==============================] - 0s 1ms/step - loss: 0.5911 - accuracy: 0.7691
Epoch 10/50
27/27 [==============================] - 0s 1ms/step - loss: 0.5820 - accuracy: 0.7727
Epoch 11/50
27/27 [

&lt;tensorflow.python.keras.callbacks.History at 0x1b8cd973550&gt;

In [42]:
# Test the model against the test set
y_pred = model.predict_classes(X_test)
print(metrics.accuracy_score(y_test,y_pred))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) &gt; 0.5).astype(&quot;int32&quot;)`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
0.7751196172248804
